In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction import text  
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [2]:
data = pd.read_csv('spam.csv') 
data = data[['Label','EmailText']]
data.dropna(axis='index',how='any',inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5587 entries, 0 to 5587
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Label      5587 non-null   object
 1   EmailText  5587 non-null   object
dtypes: object(2)
memory usage: 130.9+ KB


In [3]:

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
X_train=train_set['EmailText']
Y_train=train_set['Label']
X_test=test_set['EmailText']
Y_test=test_set['Label']

In [4]:
vectorizer = text.CountVectorizer(stop_words="english")
counts = vectorizer.fit_transform(X_train.values)
spam_nb = MultinomialNB()

In [5]:
spam_nb.fit(counts,Y_train.values)
#spam_nb.score(counts,Y_train.values)

MultinomialNB()

In [6]:
examples = vectorizer.transform(X_test.values)
predicted = spam_nb.predict(examples)

In [7]:
prediction_bool = pd.Series(predicted==Y_test.values)
true_count = prediction_bool[prediction_bool==True].count()
false_count = prediction_bool[prediction_bool==False].count()
(100*true_count)/(true_count+false_count)

98.74776386404294